In [1]:
!pip install huggingface_hub

In [15]:
# 1. Install the dotenv library
!pip install python-dotenv huggingface_hub

import os
from dotenv import load_dotenv
from huggingface_hub import login

# 2. Load the .env file
# If the file is in /content/vision/my.env, use:
load_dotenv('/content/my.env')

# 3. Retrieve and use the token
hf_token = os.getenv('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Error: HF_TOKEN not found in .env file.")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Successfully logged in to Hugging Face!


In [3]:
from __future__ import annotations

import argparse
from pathlib import Path
import zipfile

from huggingface_hub import snapshot_download


def parse_args(argv=None) -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description="Download the OpenGVLab/MVBench dataset from Hugging Face."
    )
    parser.add_argument(
        "--output-dir",
        type=Path,
        default=Path("mvbench_data"),
        help="Where to store the downloaded dataset files.",
    )
    parser.add_argument(
        "--repo-id",
        default="OpenGVLab/MVBench",
        help="Hugging Face dataset repo ID.",
    )
    parser.add_argument(
        "--revision",
        default=None,
        help="Optional git revision (branch/tag/commit).",
    )
    parser.add_argument(
        "--allow-patterns",
        default=None,
        help=(
            "Comma-separated list of glob patterns to include. "
            "Leave empty to download everything."
        ),
    )
    parser.add_argument(
        "--unzip-videos",
        action="store_true",
        help="Unzip all .zip files under the 'video' folder after download.",
    )
    return parser.parse_known_args(argv)[0]


def main() -> None:
    args = parse_args()
    output_dir = args.output_dir.resolve()
    output_dir.mkdir(parents=True, exist_ok=True)

    allow_patterns = None
    if args.allow_patterns:
        allow_patterns = [p.strip() for p in args.allow_patterns.split(",") if p.strip()]

    # Download all files in the dataset repository (full snapshot).
    snapshot_download(
        repo_id=args.repo_id,
        repo_type="dataset",
        revision=args.revision,
        local_dir=str(output_dir),
        local_dir_use_symlinks=False,
        allow_patterns=allow_patterns,
    )

    if args.unzip_videos:
        video_dir = output_dir / "video"
        if video_dir.exists():
            zip_files = sorted(video_dir.rglob("*.zip"))
            for zip_path in zip_files:
                extract_to = zip_path.parent
                with zipfile.ZipFile(zip_path, "r") as zf:
                    zf.extractall(extract_to)
        else:
            print(f"Video folder not found at: {video_dir}")

    print(f"MVBench dataset downloaded to: {output_dir}")


if __name__ == "__main__":
    main()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

MVBench dataset downloaded to: /content/mvbench_data


In [4]:
# Change directory to the video folder
%cd /content/mvbench_data/video

# Unzip all files quietly (-q) so the output doesn't lag your browser
# and delete the zip files afterward (-j can be used if you want to flatten structure)
!unzip -q "*.zip"

# Optional: Remove the zip files to save space once extracted
!rm *.zip

/content/mvbench_data/video

11 archives were successfully processed.


In [23]:
!git clone https://github.com/s973343/vision.git

Cloning into 'vision'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 23 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 25.05 KiB | 3.13 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [20]:
!pwd

/content/mvbench_data/video


In [24]:
%cd /content/vision

/content/vision


In [25]:
!pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-de_nsonc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-de_nsonc
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
!python /content/vision/batch_mvbench.py --json-dir /content/mvbench_data/json --video-dir /content/mvbench_data/video

Streaming output truncated to the last 5000 lines.
    title           : (C) 2017 Twenty Billion Neurons GmbH, 20BN-Someting-Something-Dataset V2
    encoder         : Lavf56.40.101
  Duration: 00:00:02.00, start: 0.000000, bitrate: 165 kb/s
  Stream #0:0: Video: vp9 (Profile 0), yuv420p(tv), 360x240, SAR 1:1 DAR 3:2, 12 fps, 12 tbr, 1k tbn, 1k tbc (default)
Output #0, mp3, to 'data/audio/mvbench/action_antonym_171804.mp3':
Output file #0 does not contain any stream
Error: ffmpeg error (see stderr output for detail)
Elapsed: 0.10s

[22] action_antonym | 204524
--- Starting Phase 1: Ingestion & Pre-processing ---
-> Extracting and Transcribing Audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --ena

In [13]:
%cd /content/vision

/content/vision


In [14]:
!pwd

/content/vision
